<h1> Calcul de $\pi$ </h1>

<h2> Position du problème </h2>

On se propose d'étudier différentes méthodes pour calculer les décimales du nombre $\pi$.

<h3> Convergence d'une suite </h3>

<h4> Méthode d'Archimède </h4>

Le premier calcul mathématique de $\pi$ remonte à Archimède de Syracuse (287-212 avant J.-C.). Celui-ci reposait sur un encadrement du périmètre du cercle par ceux de polygones réguliers inscrit et circonscrits. Soit $p_n$ le périmètre d'un polygone régulier à $n$ côtés inscrit dans un cercle de diamètre unité et $p'_n$ celui d'un polygone régulier à $n$ côtés circonscrit au même cercle. En utilisant l'inégalité $p_n<\pi<p'_n$ pour $n=6\times 2^k$, on peut obtenir une approximation de $\pi$ avec:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$p_n=n.u_n$   $\Big ($où $u_n=\sin{\pi\over n}$, $u_6={1\over2}$ et $u_{2n}=\sqrt{{1-\sqrt{1-u_n^2}}\over2}$ $\Big)$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; et $p'_n=n.u'_n$ $\Big ($où $u'_n=\tan{\pi\over n}$, $u'_6={1\over\sqrt{3}}$ et $u'_{2n}={u_n\over\sqrt{1-u_n^2}}$ $\Big)$.

Programmer le calcul de la suite $u_n$ par une méthode itérative et par une méthode récursive.

Pour évaluer la vitesse de convergence de cette suite, tracer $\log(\pi-p_n)$ en fonction de $n$.

In [1]:
#use "topfind";;
#require "plplot";;
open Plplot;;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/usr/lib/ocaml/plplot: added to search path
/usr/lib/ocaml/plplot/plplot.cma: loaded


In [2]:
let range debut fin =
   let rec range i acc =
     if i=fin then List.rev acc
     else range (i+1) (i::acc) in
   range debut [];;

val range : int -> int -> int list = <fun>

In [3]:
let pi =atan 1. *. 4.;;

let rec u n =
    if n=6 then 0.5 else
    sqrt((1.-.sqrt(1.-. (u (n/2))**2.))/.2.);;

let archimede n =
    float_of_int n*.(u n);;

let erreur n =
    pi-.(archimede n);;

let rec pow x n =
    if n=0 then 1 else
    if n=1 then x else 
    if n mod 2 = 0 then pow (x*x) (n/2)
    else x*(pow (x*x) (n/2));;

let liste_n=range 0 10 in
let f n = log10 (erreur (6*(pow 2 n))) in 
let liste_erreur = List.map f liste_n in

    let xs = Array.of_list (List.map float_of_int liste_n) in
    plsdev ("png");
    plsfnam("image_archimede.png");
    plscolbg 255 255 255; (* définit la couleur n°0 *)
    plscol0 15 0 0 0; (* définit la couleur n°15 *)
    plinit ();
    plcol0 15;
    plenv 0. 9. (-7.) (-1.) 0 0;
    (* xmin xmax ymin ymax x0 y0 *)
    let ys = Array.of_list liste_erreur in
    plcol0 15;
    plpoin xs ys 1; (* 1 = num du symbole *)
    
    plcol0 15;
    plmtex "b" 3.0 0.5 0.5 "n";
    plmtex "l" 5.0 0.5 0.5 "log (pi-Archimede(n))";
    plend ();;

val pi : float = 3.14159265358979312

val u : int -> float = <fun>

val archimede : int -> float = <fun>

val erreur : int -> float = <fun>

val pow : int -> int -> int = <fun>

- : unit = ()

<img src="./image_archimede.png" width=750 />

<h4> Méthode de Monte-Carlo</h4>

Tirer au sort un couple $(x_i,y_i)$ de nombres compris entre 0 et 1.

Evaluer $r_i=\sqrt{x_i^2+y_i^2}$. Si $r_i\le 1$, alors incrémenter un compteur $c$.

Réitérer le calcul pour un très grand nombre $n$ de couples $\{(x_i,y_i)\}$.

Tracer $u_n=4\times c/n$ en fonction de $n$.

In [4]:
let tirage_max = 10000 in
let liste_n = range 1 tirage_max in

let rec monte_carlo tirage_num liste inclus =
    if tirage_num = tirage_max then List.rev liste
    else begin
        if (Random.float 1.)**2.+.(Random.float 1.)**2.<=1. then
        monte_carlo (tirage_num+1) ((float_of_int(4*(inclus+1))/.(float_of_int tirage_num))::liste) (inclus+1)
        else
        monte_carlo (tirage_num+1) ((float_of_int(4*(inclus))/.(float_of_int tirage_num))::liste) inclus
        end in
    
let liste = monte_carlo 1 [] 0 in

    let xs = Array.of_list (List.map float_of_int liste_n) in
    plsdev ("png");
    plsfnam("image_monte_carlo.png");
    plscolbg 255 255 255; (* définit la couleur n°0 *)
    plscol0 15 0 0 0; (* définit la couleur n°15 *)
    plinit ();
    plcol0 15;
    plenv 0. 10000. 2.5 3.6 0 0;
    (* xmin xmax ymin ymax x0 y0 *)
    let ys = Array.of_list liste in
    plcol0 15;
    plline xs ys;
    plcol0 1;
    plline [|0.;10000.|] [|pi;pi|];
    plcol0 15;
    plmtex "b" 3.0 0.5 0.5 "n tirages";
    plmtex "l" 5.0 0.5 0.5 "Monte Carlo avec n tirages";
    let opt_array = [| [PL_LEGEND_LINE];[PL_LEGEND_LINE]|] in
    let array_colors = [|15;1|] in 
    let text_colors = array_colors in
    let text = [|"Monte-Carlo";"pi"|] in
    let line_colors = array_colors in
    let line_styles = [|1;1|] in
    let line_widths = [|1.;1.|] in
    (* note from the above opt_array the first symbol (and box) indices
       do not matter *)

    (* Second legend entry. *)
    let symbol_colors = [|15;1|] in
    let symbol_scales = [|1.;1.|] in
    let symbol_numbers = [|1;1|] in
    let symbols = [|"";""|]  in
    (* from the above opt_arrays we can completely ignore everything
       to do with boxes *)

    plscol0a 14 32 32 32 0.10;
    ignore (
      pllegend [PL_LEGEND_BACKGROUND; PL_LEGEND_BOUNDING_BOX] []
        0.0 0.0 0.1 14
        1 1 0 0
        opt_array
        1.0 1.0 2.0
        1.0 text_colors text
        [||] [||] [||] [||]
        line_colors line_styles line_widths
        symbol_colors symbol_scales symbol_numbers symbols
    );
    plend ();

- : unit = ()

<img src="./image_monte_carlo.png" width=750 />

Pour la suite, on a besoin de connaître $\pi$ avec un grande précision.

Pour cela, on utilisera la bibliothèque <b>Num</b>

In [5]:
#require "num";;
open Num;;
let pi_500 = Num.num_of_string("31415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170679821480865132823066470938446095505822317253594081284811174502841027019385211055596446229489549303819644288109756659334461284756482337867831652712019091456485669234603486104543266482133936072602491412737245870066063155881748815209209628292540917153643678925903600113305305488204665213841469519415116094330572703657595919530921861173819326117931051185480744623799627495673518857527248912279381830119491") // 
Num.num_of_string("1"^(String.make 499 '0'));;
float_of_num pi_500

/usr/lib/ocaml/nums.cma: loaded
/usr/lib/ocaml/num-top: added to search path
/usr/lib/ocaml/num-top/num_top.cma: loaded


val pi_500 : Num.num =
 <num 31415926535897932384626433832795028841971693993751058209749445923078164062862089986280348253421170679821480865132823066470938446095505822317253594081284811174502841027019385211055596446229489549303819644288109756659334461284756482337867831652712019091456485669234603486104543266482133936072602491412737245870066063155881748815209209628292540917153643678925903600113305305488204665213841469519415116094330572703657595919530921861173819326117931051185480744623799627495673518857527248912279381830119491/10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000>

- : float = 3.14159265359

In [6]:
let sqrt_2 = Num.num_of_string("14142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727350138462309122970249248360558507372126441214970999358314132226659275055927557999505011527820605714701095599716059702745345968620147285174186408891986095523292304843087143214508397626036279952514079896872533965463318088296406206152583523950547457502877599617298355752203375318570113543746034084988471603868999706990048150305440277903164542478230684929369186215805784631115966687130130156185689872372") //
Num.num_of_string("1"^(String.make 499 '0'));;
float_of_num sqrt_2

val sqrt_2 : Num.num =
 <num 3535533905932737622004221810524245196424179688442370182941699344976831196155267597125968835819103931837534615577280742562312090139626843031610303742749839578533056664818763981889499876252881955151428675273899929014925686336492155036821293546602222996523880823076210771785803627099406509069988128519974218133491365829522074101551538145880987636864375719399904324588938050843829642528385936508521247117900967249926747512037576360069475791135619557671232342296553951446157778991671782532539046422468093/2500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000>

- : float = 1.41421356237

<h4> Convergence d'une série</h4>

<h5> Série $\textrm{Arctan}$</h5>

D'après la formule de J. Gregory (1638-1675): $\textrm{Arctan} (x)=\displaystyle\sum_{k=0}^{+\infty}{(-1)^k.x^{2k+1}\over2k+1}$.
Lorsque l'on arrête la série au rang $n$, l'erreur commise est inférieure ou de l'ordre de ${|x|^{2n+3}\over2n+3}$.

On en déduit la formule de Leibniz: ${\pi\over4}=\textrm{Arctan} 1=\displaystyle\sum_{k=0}^{+\infty}{(-1)^k\over2k+1}$.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Programmer le calcul de la série $S_n=4\times\displaystyle\sum_{k=0}^{n}{(-1)^k\over2k+1}$.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Pour évaluer la vitesse de convergence de cette suite, tracer $\log(|\pi-S_n|)$ en fonction de $n$.

In [7]:
(* Arctan *)
let serie_ArcTan x n =
    let moins_un_puissance_k k =
    if k mod 2 = 0 then Num.Int(1) else Num.Int(-1) in
    
    let rec calc k sum =
    if k=n then sum
    else calc (k+1) (sum +/ ( (moins_un_puissance_k k) */ (x **/ Num.Int(2*k+1))) // Num.Int(2*k+1)) in 
    calc 0 (Num.Int(0));;

val serie_ArcTan : Num.num -> int -> Num.num = <fun>

In [8]:
let erreur n = let e = (pi_500 -/ Num.Int(4) */ (serie_ArcTan (Num.Int 1) n)) in
               if e</ (Num.Int 0) then (Num.minus_num e) else e;;
        
let liste_n = range 1 500;;
let f i = log10 (float_of_num (erreur i)) in
let liste_erreur = List.map f liste_n in

    let xs = Array.of_list (List.map float_of_int liste_n) in
    plsdev ("png");
    plsfnam("image_arctan.png");
    plscolbg 255 255 255; (* définit la couleur n°0 *)
    plscol0 15 0 0 0; (* définit la couleur n°15 *)
    plinit ();
    plcol0 15;
    plenv 0. 500. (-3.) 0. 0 0;
    (* xmin xmax ymin ymax x0 y0 *)
    let ys = Array.of_list liste_erreur in
    plcol0 15;
    plline xs ys;
    plmtex "b" 3.0 0.5 0.5 "n";
    plmtex "l" 5.0 0.5 0.5 "log10 ( pi - 4*(Arctan 1 à l'ordre n) )";
    plend ();

val erreur : int -> Num.num = <fun>

val liste_n : int list =
 [1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21;
 22; 23; 24; 25; 26; 27; 28; 29; 30; 31; 32; 33; 34; 35; 36; 37; 38; 39;
 40; 41; 42; 43; 44; 45; 46; 47; 48; 49; 50; 51; 52; 53; 54; 55; 56; 57;
 58; 59; 60; 61; 62; 63; 64; 65; 66; 67; 68; 69; 70; 71; 72; 73; 74; 75;
 76; 77; 78; 79; 80; 81; 82; 83; 84; 85; 86; 87; 88; 89; 90; 91; 92; 93;
 94; 95; 96; 97; 98; 99; 100; 101; 102; 103; 104; 105; 106; 107; 108; 109;
 110; 111; 112; 113; 114; 115; 116; 117; 118; 119; 120; 121; 122; 123; 124;
 125; 126; 127; 128; 129; 130; 131; 132; 133; 134; 135; 136; 137; 138; 139;
 140; 141; 142; 143; 144; 145; 146; 147; 148; 149; 150; 151; 152; 153; 154;
 155; 156; 157; 158; 159; 160; 161; 162; 163; 164; 165; 166; 167; 168; 169;
 170; 171; 172; 173; 174; 175; 176; 177; 178; 179; 180; 181; 182; 183; 184;
 185; 186; 187; 188; 189; 190; 191; 192; 193; 194; 195; 196; 197; 198; 199;
 200; 201; 202; 203; 204; 205; 206; 207; 208; 209; 210; 211; 212; 213; 214;
 215; 216; 217; 218; 219; 220; 221; 222; 223; 224; 225; 226; 227; 228; 229;
 230; 231; 232; 233; 234; 235; 236; 237; 238; 239; 240; 241; 242; 243; 244;
 245; 246; 247; 248; 249; 250; 251; 252; 253; 254; 255; 256; 257; 258; 259;
 260; 261; 262; 263; 264; 265; 266; 267; 268; 269; 270; 271; 272; 273; 274;
 275; 276; 277; 278; 279; 280; 281; 282; 283; 284; 285; 286; 287; 288; 289;
 290; 291; 292; 293; 294; 295; 296; 297; 298; 299; ...]

- : unit = ()

<img src="./image_arctan.png" width=750 />

Cette série convergeant assez lentement (car $x=1$), on peut - pour accélérer la convergence - utiliser des
combinaisons de fonctions $\textrm{Arctan} x$ avec $x\ll1$. Programmer une série utilisant la formule de Gauss:
$\pi=48.\textrm{Arctan}{1\over18}+32.\textrm{Arctan}{1\over57}-20.\textrm{Arctan}{1\over239}$

In [9]:
let gauss n =
    let un_sur_dixhuit = (Num.Int 1) // (Num.Int 18) in
    let un_sur_cinquantesept = (Num.Int 1) // (Num.Int 57) in
    let un_sur_deuxcenttrenteneuf = (Num.Int 1) // (Num.Int 239) in
    (Num.Int 48 */ (serie_ArcTan un_sur_dixhuit n) +/
     Num.Int 32 */ (serie_ArcTan un_sur_cinquantesept n) -/
     Num.Int 20 */ (serie_ArcTan un_sur_deuxcenttrenteneuf n))

let erreur n = let e =(pi_500 -/ (gauss n)) in
               if e</ (Num.Int 0) then Num.minus_num e else e;;

let liste_n = range 1 100 in
let f i = log10 (Num.float_of_num (erreur i)) in
let liste_erreur = List.map f liste_n in


    let xs = Array.of_list (List.map float_of_int liste_n) in
    plsdev ("png");
    plsfnam("image_gauss.png");
    plscolbg 255 255 255; (* définit la couleur n°0 *)
    plscol0 15 0 0 0; (* définit la couleur n°15 *)
    plinit ();
    plcol0 15;
    plenv 0. 100. (-300.) 0. 0 0;
    (* xmin xmax ymin ymax x0 y0 *)
    let ys = Array.of_list liste_erreur in
    plcol0 15;
    plline xs ys;
    plmtex "b" 3.0 0.5 0.5 "n";
    plmtex "l" 5.0 0.5 0.5 "log10 ( pi - (Gauss à l'ordre n) )";
    plend ();

val gauss : int -> Num.num = <fun>

val erreur : int -> Num.num = <fun>

- : unit = ()

<img src="./image_gauss.png" width=750 />

<h5> Série de Ramanujan</h5>

Au début du vingtième siècle, S. Ramanujan, mathématicien autodidacte indien, proposa la série:

$\displaystyle{1\over\pi}={2.\sqrt{2}\over9801}\times\sum_{k=0}^{+\infty}{(4k)!.(1103+26390.k)\over(n!)^4.393^{4n}}$.

Programmer le calcul de la série $S'_n={9801\over2.\sqrt{2}}\times{1\over\displaystyle\sum_{k=0}^{n}{(4k)!.(1103+26390.k)\over(n!)^4.393^{4n}}}$.

Cette série converge très vite: vérifier qu'au bout de 2 termes, on a déjà $\pi$ à 8 décimales.
Mais les termes supplémentaires ne permettent de gagner qu'une précision relativement faible.

Evaluer l'évolution de la précision $P_n=\pi-S'_n$ en traçant $\log(|P_{n+1}-P_{n}|)$ en fonction de $n$.

In [10]:
let factorial n =
    let rec factorial n m =
       if (n >/ m) then factorial m n
       else if m =/ (Num.Int 0) then Num.Int 1
       else if n =/ m then n
       else (factorial n (Num.quo_num (n+/m) (Num.Int 2))) */ ( factorial(Num.quo_num (n+/m) (Num.Int 2) +/ (Num.Int 1)) m) in
    factorial (Num.Int n) (Num.Int 0);;

val factorial : int -> Num.num = <fun>

In [12]:
let s n =
    let sum= ref (Num.Int 0) in
    for k = 0 to (n-1) do
        let num=factorial(4*k) */ (Num.Int (1103 + 26390*k)) in
        let den=(factorial(k)**/(Num.Int 4)) */ (Num.Int 393)**/(Num.Int (4*k)) in
        sum:=!sum +/ (num//den);
    done; !sum;;

let ramanujan n = (Num.Int 9801)//(Num.Int 2 */ sqrt_2 */ (s n));;

let erreur n = let e =(pi_500 -/ (ramanujan n)) in
               if e</ (Num.Int 0) then Num.minus_num e else e;;

let liste_n = range 1 50;;
let f i = log10 (Num.float_of_num (erreur i)) in
let liste_erreur = List.map f liste_n in

    let xs = Array.of_list (List.map float_of_int liste_n) in
    plsdev ("png");
    plsfnam("image_ramanujan.png");
    plscolbg 255 255 255; (* définit la couleur n°0 *)
    plscol0 15 0 0 0; (* définit la couleur n°15 *)
    plinit ();
    plcol0 15;
    plenv 0. 50. (-20.) 0. 0 0;
    (* xmin xmax ymin ymax x0 y0 *)
    let ys = Array.of_list liste_erreur in
    plcol0 15;
    plline xs ys;
    plmtex "b" 3.0 0.5 0.5 "n";
    plmtex "l" 5.0 0.5 0.5 "log10 ( pi - (Ramanujan à l'ordre n) )";
    plend ();

val s : int -> Num.num = <fun>

val ramanujan : int -> Num.num = <fun>

val erreur : int -> Num.num = <fun>

val liste_n : int list =
 [1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21;
 22; 23; 24; 25; 26; 27; 28; 29; 30; 31; 32; 33; 34; 35; 36; 37; 38; 39;
 40; 41; 42; 43; 44; 45; 46; 47; 48; 49]

- : unit = ()

<img src="./image_ramanujan.png" width=750 />

In [13]:
let liste_n = range 0 40;;
let array_precision = Array.make 40 0. in
for i=1 to 39 do
    array_precision.(i)<- log10( abs_float( Num.float_of_num (ramanujan(i+1)-/ (ramanujan i))))
done;

    let xs = Array.of_list (List.map float_of_int liste_n) in
    plsdev ("png");
    plsfnam("image_ramanujan2.png");
    plscolbg 255 255 255; (* définit la couleur n°0 *)
    plscol0 15 0 0 0; (* définit la couleur n°15 *)
    plinit ();
    plcol0 15;
    plenv 0. 40. (-350.) 0. 0 0;
    (* xmin xmax ymin ymax x0 y0 *)
    let ys = array_precision in
    plcol0 15;
    plline xs ys;
    plmtex "b" 3.0 0.5 0.5 "n";
    plmtex "l" 5.0 0.5 0.5 "log10( (Ramanujan ordre n+1) - (Ramanujan ordre n) )";
    plend ();

val liste_n : int list =
 [0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20;
 21; 22; 23; 24; 25; 26; 27; 28; 29; 30; 31; 32; 33; 34; 35; 36; 37; 38;
 39]

- : unit = ()

<img src="./image_ramanujan2.png" width=750 />